# **[Projeto Observatório Social - Prefeitura Municipal de Indaial | SC]**
<font color=white> *Usando o Python para capturar os dados e inseri-los em um Banco de Dados (SQL Server) para posterior análises.

### Sobre o projeto


Entender a eficiência do legislativo da cidade de Indaial em Santa Catarina é um ponto chave para entidades sociais (que fiscalizam e avaliam políticas publicas), especialmente àquelas que tratam do patrimônio publico e de temas pertinentes à vida social. Com base nesta problemática, foi desenvolvido uma solução que visa captar dados do site câmara legislativa de Indaial com vistas a entender o estado da arte do legislativo local. 
Para isso, foi realizado uma coleta das preposições (*Webscrapping*), por intermédio do `Python`, de projetos dos vereadores, com armazenamento incremental em um banco de dados(`SQL Server`) e suas análise com uso do `Power BI`.
Considere que preposição é “[…] é toda matéria sujeita à deliberação do Plenário, qualquer que seja o seu objeto.” Regimento Interno da Câmara Municipal de Indaial no seu artigo 113

### Questão do negócio:

Como se comportam os vereadores da cidade de Indaial é algo fundamental para entidades fiscalizadoras, não somente na produção de leis, mas também na construção de pautas que sirvam à cidade. Assim, os assuntos tratados, para além do volume de proposições é fundamental para o bom funcionamento da camara e suas relações com o executivo local.

### Fonte dos dados:

Site da Câmara de Indaial: https://www.camaraindaial.sc.gov.br/pg/proposicoes

### Tecnologias utilizadas:

a) Foi utilizado o **Python** para realizar a busca e captura das preposições no site da câmara

b) O **SQL Server** para criação do banco de dados com armazenamento local

c) O **Power BI** na construção do Dashboard na modelagem e tratamento dos dados

### Setar o ambiente

Para replicar o projeto você precisará instalar as seguintes bibliotecas no Python:
>> - python -m pip install pip==20.3.1
>>>>> - pip install jupyterlab==2.2.9
>>>>> - pip install numpy==1.18.3
>>>>> - pip install pandas==1.1.5
>>>>> - pip install matplotlib==3.3.3
>>>>> - pip install seaborn==0.11.0
>>>>> - pip install openpyxl==3.0.5
>>>>> - pip install xlrd==1.2.0
>>>>> - pip install folium==0.12.1
>>>> - Específicas para webscraping nesse projeto
>>>>> - pip install beautifulsoup4==4.9.3
>>>>> - pip install html5lib==1.1
>>>>> - pip install requests==2.25.1
>>>>> - pip install lxml==4.6.2


E os seguintes programas:

>>- PowerBI (versão mais recente) - https://powerbi.microsoft.com/pt-br/downloads/
- SQL Server Developer Edition (versão mais recente) - Versão desenvolvedor - https://www.microsoft.com/pt-br/sql-server/sql-server-downloads
    - SSMS - Microsoft Management Studio (versão mais recente) - [https://docs.microsoft.com/pt-br/sql/ssms/download-sql-server-management-studio-ssms?view=sql-server-](https://docs.microsoft.com/pt-br/sql/ssms/download-sql-server-management-studio-ssms?view=sql-server-ver15)
- Visual Studio - https://code.visualstudio.com/-
    - Extensão do Jupyter no Visual Studio (versão mais recente)

In [1]:
# Importando as bibliotecas para o projeto

import time
import pandas as pd
import bs4
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen

In [3]:
# Declarando o agente

agente = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': agente}

In [4]:
# Função que faz consulta nas paginas da Web

def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass

In [5]:
# Função que captura os itens da pagina e possibilita o seu uso (objeto)

def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')    
    return soup

In [8]:
#O conteudo disposto na pagina em formato de tabela, de fato não é uma tabela. Por isso é inviavel a utilização da biblioteca Pandas.
# Para solucionar esta questão, criei um dicionario fazer um conjunto chave:valor {-:-}
def Cabecalho(html):
    dt = html.find_all('dt')
    dd = html.find_all('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text() 
        dic[x] = y
    return dic

In [9]:
#Automatizando a busca nas paginas das preposicoes e dos anos
def Conteudo(proposicao, ano):
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url)
    dic = Cabecalho(html)
    dic['Proposição'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text()
    return dic

In [10]:
#Função que cria a tabela com base em uma quantidade de preposicoes e um nao especifico


def TabelaResultados(inicar_em, quantidade, ano, erros_admissiveis, segundos_espera):

    ultima_consulta = inicar_em + quantidade - 1

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while inicar_em <= ultima_consulta and erros <= erros_admissiveis:

        try:
            x = Conteudo(inicar_em,ano)
            lista = lista + [Conteudo(inicar_em,ano)]
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        inicar_em+=1 
        i+=1


    return pd.DataFrame(lista)

In [11]:
teste = TabelaResultados(500,3,2021,2,0.2)
teste

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto
0,08/04/2021,08/04/2021,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,500,2021,"O vereador que está subscreve, no uso das atri..."
1,08/04/2021,08/04/2021,Proposição Aprovada,"Pontes, Bueiros, boca de lobo",Vereador Flávio Augusto Ferri Molinari.,501,2021,"O vereador que está subscreve, no uso das atri..."
2,08/04/2021,08/04/2021,Proposição Aprovada,Manutenção de via pública,Vereador Fernanda dos Santos.,502,2021,"A vereadora que esta subscreve, no uso das atr..."


In [12]:
#Importando a biblioteca que fará a conexão com o SGBD
import pyodbc

In [13]:
#Realizando a conexão com o banco de dados MySQL
conn = pyodbc.connect('Trusted_Connection=yes',
                        driver = '{ODBC Driver 17 for SQL Server}',
                        server = 'BRUNO', # eu renomeie o localhost para BRUNO(apenas gosto meu)
                        database = 'Indaial')
query = '''
    select
    *
    from Proposicoes

'''

sql_query = pd.read_sql_query(query,conn)
sql_query

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,VereadorHenrique Fritz.,1,1996,construção de Escola de 1º grau no Bairro Nova...
1,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,VereadorHenrique Fritz.,2,1996,construção de Escola nas imediações dos Loteam...
2,1996-02-22,1996-02-22,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorHenrique Fritz.,3,1996,alargamento da Rua ID 90 ...
3,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,VereadorHenrique Fritz.,4,1996,prolongamento da rede d'água na Rua Reinhold S...
4,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,VereadorHenrique Fritz.,5,1996,Prolongamento de rede d'água na Rua Lorenz até...
...,...,...,...,...,...,...,...,...
29135,1900-01-01,1900-01-01,,,,1140,2022,Usamos cookies e tecnologias semelhantes para ...
29136,1900-01-01,1900-01-01,,,,1142,2022,Usamos cookies e tecnologias semelhantes para ...
29137,1900-01-01,1900-01-01,,,,1146,2022,Usamos cookies e tecnologias semelhantes para ...
29138,1900-01-01,1900-01-01,,,,1149,2022,Usamos cookies e tecnologias semelhantes para ...


In [14]:
base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
base

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto


In [15]:
#Tratando os casos que estarão vazios na conexão com o Banco
base.fillna('')

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto


In [22]:
base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
teste = base.append(teste).fillna('')

conn = pyodbc.connect('Trusted_Connection=yes',
                        driver = '{ODBC Driver 17 for SQL Server}',
                        server = 'BRUNO', # eu renomeie o localhost para BRUNO(apenas gosto meu)
                        database = 'Indaial')
cursor = conn.cursor()

for index, row in teste.iterrows():
    
    cursor.execute('''
    
        INSERT INTO Proposicoes (
            DataReuniao,
            DataDeliberacao,
            Situacao,
            Assunto,
            Autor,
            Proposicao,
            Ano,
            Texto
        ) 
        values(?,?,?,?,?,?,?,?)''', # mesma quantidade de variáveis

        row['Reunião'], 
        row['Deliberação'], 
        row['Situação'], 
        row['Assunto'], 
        row['Autor'], 
        row['Proposição'], 
        row['Ano'], 
        row['Texto']

    )

conn.commit()
cursor.close()

In [19]:
#Definindo na conexão funções do SQL

conn = pyodbc.connect('Trusted_Connection=yes',
                        driver = '{ODBC Driver 17 for SQL Server}',
                        server = 'BRUNO', 
                        database = 'Indaial')
query = '''
    select
    *
    from Proposicoes

'''

sql_query = pd.read_sql_query(query,conn)
sql_query

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,VereadorHenrique Fritz.,1,1996,construção de Escola de 1º grau no Bairro Nova...
1,1996-02-22,1996-02-22,Proposição Aprovada,Serviços e Obras,VereadorHenrique Fritz.,2,1996,construção de Escola nas imediações dos Loteam...
2,1996-02-22,1996-02-22,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",VereadorHenrique Fritz.,3,1996,alargamento da Rua ID 90 ...
3,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,VereadorHenrique Fritz.,4,1996,prolongamento da rede d'água na Rua Reinhold S...
4,1996-02-22,1996-02-22,Proposição Aprovada,Rede de Água / Esgoto / Pluvial,VereadorHenrique Fritz.,5,1996,Prolongamento de rede d'água na Rua Lorenz até...
...,...,...,...,...,...,...,...,...
29135,1900-01-01,1900-01-01,,,,1140,2022,Usamos cookies e tecnologias semelhantes para ...
29136,1900-01-01,1900-01-01,,,,1142,2022,Usamos cookies e tecnologias semelhantes para ...
29137,1900-01-01,1900-01-01,,,,1146,2022,Usamos cookies e tecnologias semelhantes para ...
29138,1900-01-01,1900-01-01,,,,1149,2022,Usamos cookies e tecnologias semelhantes para ...


In [20]:
#Automatizando a criação das funções no SQL no Python
def SQLSelect(query):
    
    conn = pyodbc.connect('Trusted_Connection=yes',
                    driver = '{ODBC Driver 17 for SQL Server}',
                    server = 'BRUNO', 
                    database = 'Indaial')

    out = pd.read_sql_query(query,conn)
    return out

In [23]:
SQLSelect('''
    select 
        *
    from Proposicoes Where Ano = 2023
''')

,DataReuniao,DataDeliberacao,Situacao,Assunto,Autor,Proposicao,Ano,Texto
0,2023-02-02,2023-02-02,Proposição Despachada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Remir José de Faveri.,1,2023,"O vereador que esta subscreve, no uso das atri..."
1,2023-02-02,2023-02-02,Proposição Despachada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Jonas Luiz de Lima.,2,2023,"O Vereador que esta subscreve, no uso das atri..."
2,2023-02-02,2023-02-02,Proposição Despachada,Educação,Vereador Remir José de Faveri.,5,2023,"O vereador que esta subscreve, no uso das atri..."
3,2023-02-02,2023-02-02,Proposição Despachada,Rede de Água / Esgoto / Pluvial,"Vereador Jonas Luiz de Lima, Remir José de Fav...",6,2023,"Os vereadores abaixo firmados, após ouvido o P..."
4,2023-02-02,2023-02-02,Proposição Despachada,Manutenção de via pública,Vereador Remir José de Faveri.,7,2023,"O vereador que esta subscreve, no uso das atri..."
...,...,...,...,...,...,...,...,...
2217,1900-01-01,1900-01-01,,,,2218,2023,Usamos cookies e tecnologias semelhantes para ...
2218,1900-01-01,1900-01-01,,,,2220,2023,Usamos cookies e tecnologias semelhantes para ...
2219,1900-01-01,1900-01-01,,,,2222,2023,Usamos cookies e tecnologias semelhantes para ...
2220,1900-01-01,1900-01-01,,,,2221,2023,Usamos cookies e tecnologias semelhantes para ...


In [27]:
SQLSelect('''
   select 
        maior = max(Proposicao)
    from Proposicoes
''')

,maior
0,2281


In [28]:
# Populando os dados no Banco de Dados
def SQLInsertProposicoes(TabelaProposicoes):
    
    base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes',
                driver = '{ODBC Driver 17 for SQL Server}',
                server = 'BRUNO', 
                database = 'Indaial')

    cursor = conn.cursor() #o 'agente' que faz a 'entrega' dos dados no banco de dados, tal como um 'motorista'.

    for index, row in TabelaProposicoes.iterrows():

        cursor.execute('''

            INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            ) 
            values(?,?,?,?,?,?,?,?)''', 

            row['Reunião'], 
            row['Deliberação'], 
            row['Situação'], 
            row['Assunto'], 
            row['Autor'], 
            row['Proposição'], 
            row['Ano'], 
            row['Texto']

        )

    conn.commit()
    cursor.close()

In [24]:
ano = 2021
dados = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
dados

,Proposicao
0,2282


In [30]:
dados['Proposicao']

0    2281
Name: Proposicao, dtype: int64

In [31]:
dados['Proposicao'].loc[0]

2281

In [25]:
#Função que trabalha inserindo preposições que ainda não existem no banco
def InsereProximaProposicao(ano):

    # Identifica a ultima proposição
    dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]

    # Verifica lançamento naquele ano
    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao) + 1 

    # Captura e Insere dados na tabela
    dados = Conteudo(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsertProposicoes(tabela)

In [26]:
ano = 2023
quantidade = 25
erros_admissiveis = 2
segundos_espera = 2

# erros
erros = 0

# variaveis para loop
i = 1
lista = []

while erros <= erros_admissiveis:

    try:
        InsereProximaProposicao(ano)
    except:
        erros += 1
        pass

    time.sleep(segundos_espera)

    # carregamento incremental das variáveis
    i +=1

In [28]:
SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {2023}')

,Proposicao
0,2222


In [37]:
#Observando alguns cenários como falta de energia, site fora do ar, preposição já existe, etc esta função desempenha esse papel

def BuscaGravaDadosAno(ano, quantidade = 0, erros_admissiveis = 2, segundos_espera = 6):

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while erros <= erros_admissiveis:

        try:
            InsereProximaProposicao(ano)
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        i +=1

In [38]:
ano_inicial = 2023
ano_final = 2023

for i in list(range(ano_inicial, ano_final+1)):
    print('Populando dados de: ',i)
    try:
        BuscaGravaDadosAno(i, quantidade = 2400)
    except:
        pass
    
print('Gravação de dados concluída')

Iniciando gravação dos dados do ano:  2022
Iniciando gravação dos dados do ano:  2023
